<a href="https://colab.research.google.com/github/dderaad/Hackathon-Summer-2021/blob/main/clfs_mk1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>